In [6]:
import pandas as pd
from glob import glob
import json
import os
import re

In [7]:
def get_filelst(SAVE_PATH):
    return list(set([i.replace("_0_shot.json","") for i in os.listdir(SAVE_PATH)]))

# results/korean_origin_bench/20b/00_shot.json
def _get_metric_name(v):
    metrics = ['f1', 'macro_f1', 'acc_norm', 'acc']
    for m in metrics:
        if v.get(m):
            return {
                'metric': m,
                'value': v[m],
            }

def get_df_klue(SAVE_PATH, model_name=''):
    
    data = []
    for i in ['0', '5', '10', '50']:
        shot = f'{SAVE_PATH}/{model_name}_{i}_shot.json'
        try:

            data.append(
                {   
                    # 'kobest_boolq (macro_f1)': 0.35461338879835325
                    f"{k} ({_get_metric_name(v)['metric']})": _get_metric_name(v)['value']
                    for k, v in json.load(open(shot))['results'].items()
                }
            )
        except FileNotFoundError:
            pass
    df = pd.DataFrame(data, index=[model_name]).T
    return df

In [8]:
SAVE_PATH = '../data/result'
filelst = get_filelst(SAVE_PATH)
print("file 개수 : ",len(filelst))
temp_df = []
ignore_lst = [
    '(davidkim205)_nox-solar-10.7b-v4',
    '(01-ai)_Yi-6b',
    '(yanolja)_EEVE-Korean-10.8B-v1.0',
    '(mistralai)_Mistral-7B-v0.1',
    '(beomi)_Yi-ko-6b',
    "(ours)_Mixtral-instruct_v0.1"
]

for file in filelst:  
    if file in ignore_lst:
        continue    
    if re.search("gemma",file):
        continue
    pattern = r"^\(ours\)_Synatra"
    if re.match(pattern, file):
        continue
    if re.search("llama",file):
        continue
    pattern = r"^\(hwkwon\)"
    if re.search(pattern,file):
        df = get_df_klue(SAVE_PATH,model_name=file)
        temp_df.append(df)
        continue
    if re.search("SOLAR",file):
        continue  
    df = get_df_klue(SAVE_PATH,model_name=file)
    temp_df.append(df)
    
print("일부 제거 후 file 개수 : ",len(temp_df))
all_df = pd.concat(temp_df,axis=1).T.reset_index()
all_df.rename(columns={"index":"name"},inplace=True)
all_df.to_csv("data.csv",index=False)

file 개수 :  38
일부 제거 후 file 개수 :  11


In [9]:
all_df['name'].values

array(['(ours)_kospo_mixtral_ver0.4', '(ours)_kospo_mixtral_ver0.3',
       '(hwkwon)_S-SOLAR-10.7B-v1.5',
       '(yanolja)_EEVE-Korean-Instruct-10.8B-v1.0',
       '(mistralai)_Mixtral-8x7B-Instruct-v0.1',
       '(ours)_kospo_synatra_ver0.1', '(maywell)_Synatra_base',
       '(ours)_kospo_eeve_ver0.2', '(ours)_kospo_eeve_ver0.1',
       '(mistralai)_Mixtral-8x7B-v0.1', '(ours)_kospo_T3Q_ver0.1'],
      dtype=object)

In [10]:
import pandas as pd
import pygwalker as pyg
from pygwalker import GlobalVarManager
GlobalVarManager.set_kanaries_api_key("ak-5e3a66b8e419a35a39118a6fa64387474fe2743179f8a6169eaee4637f78a3ac")
df = pd.read_csv('./data.csv')
walker = pyg.walk(df,hideDataSourceConfig=True, vegaTheme='vega')

Box(children=(HTML(value='<div id="ifr-pyg-000616d3764233d1nMZgdR3ihDCcbPH0" style="height: auto">\n    <head>…

In [15]:
import pygwalker as pyg
import pandas as pd
import streamlit.components.v1 as components
import streamlit as st
 
# Adjust the width of the Streamlit page
st.set_page_config(
    page_title="Use Pygwalker In Streamlit",
    layout="wide"
)
 
# Add Title
st.title("Use Pygwalker In Streamlit")
 
# Import your data
df = pd.read_csv("https://kanaries-app.s3.ap-northeast-1.amazonaws.com/public-datasets/bike_sharing_dc.csv")
 
# Paste the copied Pygwalker chart code here
vis_spec = """<PASTE_COPIED_CODE_HERE>"""
 
# Generate the HTML using Pygwalker
pyg_html = pyg.to_html(df, spec=vis_spec)
 
# Embed the HTML into the Streamlit app
components.html(pyg_html, height=1000, scrolling=True)

DeltaGenerator()